# 图像尺寸对训练效果的影响

In [1]:
import os
import sys
import cv2
import h5py
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import time
from datetime import datetime
from tqdm import tqdm
from utils import get_params_count

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from keras.applications import inception_v3, xception, resnet50, vgg16, vgg19
from keras.applications import InceptionV3, Xception, ResNet50, VGG16, VGG19
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.layers import Input, Dense, Dropout, Activation, Flatten, Lambda
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.models import Model
from keras.optimizers import SGD

Using TensorFlow backend.


In [2]:
def load_input(height, width, input_type):
    train_name = os.listdir('./train')
    test_name = os.listdir('./test')
    train_size = len(train_name)
    test_size = len(test_name)
    X_train = np.zeros((train_size, height, width, 3), dtype=input_type)
    X_test = np.zeros((test_size, height, width, 3), dtype=input_type)
    
    labels = pd.read_csv('labels.csv')
    breeds = list(set(labels['breed']))
    breeds.sort()
    Y_test = pd.read_csv('sample_submission.csv')

    # Labels
    Y_train = np.zeros((train_size, len(breeds)), dtype=np.uint8)
    for i in range(train_size):
        onehot = breeds.index(labels['breed'][i])
        Y_train[i][onehot] = 1

    # Train data
    for i in tqdm(range(train_size)):
        img = cv2.imread('./train/%s.jpg' % labels['id'][i])
        img = cv2.resize(img, dsize=(width, height))
        img = img[:, :, ::-1]
        X_train[i] = img

    # Test data
    for i in tqdm(range(test_size)):
        img = cv2.imread('./test/%s.jpg' % Y_test['id'][i])
        img = cv2.resize(img, dsize=(width, height))
        img = img[:, :, ::-1]
        X_test[i] = img

    print("Train: %d, Test: %d" % (train_size, test_size))
    print("Total Dog Breeds:", len(breeds))
    print('Training Data Size = %.2f GB' % (sys.getsizeof(X_train)/1024**3))
    print('Testing Data Size = %.2f GB' % (sys.getsizeof(X_test)/1024**3))
    
    return X_train, X_test, Y_train, Y_test

In [3]:
height = 299
width = 299

In [5]:
X_train, X_test, Y_train, Y_test = load_input(height, width, np.uint8)

100%|█████████████████████████████████████████████████████| 10357/10357 [00:26<00:00, 384.36it/s]


Train: 10222, Test: 10357
Total Dog Breeds: 120
Training Data Size = 2.55 GB
Testing Data Size = 2.59 GB


In [7]:
Y_train

(10222, 120)

In [21]:
def export_gap(MODEL, height, width, train, test, preprocess=None, batch_size=128):
    x = Input(shape=(height, width, 3))
    if preprocess is not None:
        x = Lambda(preprocess)(x)
    model = MODEL(include_top=False, input_tensor=x, weights='imagenet', pooling='avg')
    train_gap = model.predict(train, batch_size=batch_size)
    test_gap = model.predict(test, batch_size=batch_size)
    with h5py.File("gap_%dx%d_%s.h5" % (height, width, MODEL.__name__), 'w') as f:
        f.create_dataset('train', data=train_gap)
        f.create_dataset('test', data=test_gap)

In [22]:
export_gap(InceptionV3, height, width, X_train, X_test, inception_v3.preprocess_input)

In [23]:
train_gap = []
test_gap = []
# 'gap_InceptionV3.h5', 'gap_Xception.h5', 'gap_ResNet50.h5', 'gap_InceptionResNetV2.h5'
for gapfile in ['gap_299x299_InceptionV3.h5']:
    with h5py.File(gapfile, 'r') as f:
        train_gap.append(np.array(f['train']))
        test_gap.append(np.array(f['test']))
train_gap = np.concatenate(train_gap, axis=1)
test_gap = np.concatenate(test_gap, axis=1)
print("Number of Features:", train_gap.shape[1])

Number of Features: 2048


In [24]:
X_train_gap, X_val_gap, Y_train, Y_val = train_test_split(train_gap, Y_train, shuffle=True, test_size=0.2, 
                                                          random_state=42)

In [25]:
# Input Shape: (Batch Size, Feature Vector length)
x = Input(shape=(X_train_gap.shape[1],))
y = Dropout(0.2)(x)
y = Dense(120, activation='softmax', kernel_initializer='he_normal', name='classifier')(y)
model_gap = Model(inputs=x, outputs=y, name='GAP')
model_gap.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
print('Trainable: %d, Non-Trainable: %d' % get_params_count(model_gap))

# Prepare Callbacks for Model Checkpoint, Early Stopping and Tensorboard.
log_name = '/DogBreed-EP{epoch:02d}-LOSS{val_loss:.4f}.h5'
log_dir = datetime.now().strftime('gap_model_%Y%m%d_%H%M')
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

es = EarlyStopping(monitor='val_loss', patience=20)
mc = ModelCheckpoint(log_dir + log_name, monitor='val_loss', save_best_only=True)
tb = TensorBoard(log_dir=log_dir)

model_gap.fit(x=X_train_gap, y=Y_train, batch_size=32, epochs=20, validation_data=(X_val_gap, Y_val), 
              callbacks=[es, mc, tb])

Trainable: 245880, Non-Trainable: 0
Train on 8177 samples, validate on 2045 samples
Epoch 1/20
8177/8177 [==============================] - 1s 138us/step - loss: 2.2564 - acc: 0.6661 - val_loss: 0.9128 - val_acc: 0.8782
Epoch 2/20
8177/8177 [==============================] - 1s 111us/step - loss: 0.6235 - acc: 0.8940 - val_loss: 0.4943 - val_acc: 0.8910
Epoch 3/20
8177/8177 [==============================] - 1s 111us/step - loss: 0.3961 - acc: 0.9128 - val_loss: 0.3842 - val_acc: 0.9042
Epoch 4/20
8177/8177 [==============================] - 1s 113us/step - loss: 0.3151 - acc: 0.9215 - val_loss: 0.3414 - val_acc: 0.9007
Epoch 5/20
8177/8177 [==============================] - 1s 116us/step - loss: 0.2766 - acc: 0.9233 - val_loss: 0.3158 - val_acc: 0.9037
Epoch 6/20
8177/8177 [==============================] - 1s 119us/step - loss: 0.2520 - acc: 0.9282 - val_loss: 0.3075 - val_acc: 0.9002
Epoch 7/20
8177/8177 [==============================] - 1s 113us/step - loss: 0.2328 - acc: 0.9305 -